In [7]:
!pip install sympy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import numpy as np
import pandas as pd
from sclibrary.simplicial_complex import SimplicialComplexNetwork

In [9]:
data_folder = 'data/sample_data'

B1 = pd.read_csv(data_folder + '/B1.csv', header=None)
print(B1.shape)
B1

(5, 7)


,0,1,2,3,4,5,6
0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,-1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,-1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [10]:
B2 = pd.read_csv(data_folder + '/B2.csv', header=None)
print(B2.shape)
B2

(2, 7)


,0,1,2,3,4,5,6
0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,-1.0,0.0,1.0


In [11]:
from sclibrary.read_incidence_matrix import ReadIncidenceMatrix

data = ReadIncidenceMatrix(B1, B2)
data.get_data_summary()
print("Nodes", data.get_nodes())
print("Edges", data.get_edge_list(rank=1))
print("Triangle Edges", data.get_edge_list(rank=2))
print("Adjacency", data.get_adjacency_matrix())

edges = data.get_edge_list()
print(edges)
sc = SimplicialComplexNetwork(edges, pos=None)

number of nodes:  5
number of edges:  7
number of triangles:  2
Nodes [0, 1, 2, 3, 4]
Edges [(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4)]
Triangle Edges [[1, 2, 3], [1, 3, 4]]
Adjacency [[ 0 -1 -1  0  0]
 [ 1  0 -1 -1 -1]
 [ 1  1  0 -1  0]
 [ 0  1  1  0 -1]
 [ 0  1  0  1  0]]
[(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4), [1, 2, 3], [1, 3, 4]]


In [12]:
L1 = sc.hodge_laplacian_matrix(rank=1)
L1L = sc.lower_laplacian_matrix(rank=1)
L1U = sc.upper_laplacian_matrix(rank=1)
L1

array([[ 2.,  1., -1., -1., -1.,  0.,  0.],
       [ 1.,  2.,  1.,  0.,  0., -1.,  0.],
       [-1.,  1.,  3.,  0.,  1.,  0.,  0.],
       [-1.,  0.,  0.,  4.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  3.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  3., -1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  3.]], dtype=float32)

In [97]:
import numpy as np
import sympy


def get_harmonic_eigenvectors(hodgle_lap_mat: np.ndarray) -> tuple:
    u_h, lambda_vals = _get_eigenvectors(hodgle_lap_mat)
    return u_h, lambda_vals



def get_curl_eigenvectors(upper_lap_mat: np.ndarray) -> tuple:
    u_c, lambda_vals = _get_eigenvectors(upper_lap_mat)
    return u_c, lambda_vals


def get_gradient_eigenvectors(lower_lap_mat: np.ndarray) -> tuple:
    u_g, lambda_vals = _get_eigenvectors(lower_lap_mat)
    return u_g, lambda_vals


def _get_eigenvectors(lap_mat: np.ndarray) -> tuple:
    eigenvalues, eigenvectors = np.linalg.eig(lap_mat)
    # remove small values due to numerical errors
    tolerance = 0.1 / np.abs(eigenvalues).max()
    eigenvectors[np.abs(eigenvectors) < tolerance] = 0
    lambda_values = np.diag(eigenvalues)
    lambda_values[lambda_values < 1e-3] = 0
    # L(k) = U(k) * lambda(k) * U(k).T
    assert np.allclose(
        np.rint(eigenvectors @ lambda_values @ np.linalg.inv(eigenvectors)),
        lap_mat,
    )
    return eigenvectors, lambda_values


def get_matrix_image(matrix: np.ndarray) -> tuple:
    matrix_t = sympy.Matrix(matrix.T)
    rref_matrix, pivot_columns = matrix_t.rref()
    return rref_matrix, pivot_columns


In [103]:
u_g, lambda_vals_g = get_harmonic_eigenvectors(L1)
u_c, lambda_vals_c = get_curl_eigenvectors(L1U)
u_h, lambda_vals_h = get_gradient_eigenvectors(L1L)
lambda_vals_h

array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 1.5857865, 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 3.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 5.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 4.4142137, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ]], dtype=float32)

In [104]:
# flow geenration
flow = np.random.rand(L1.shape[0], 1)
print("flow", flow)
# analyze frequency components
flow_h = u_h.T @ flow
flow_g = u_g.T @ flow
flow_c = u_c.T @ flow
print("flow_h", flow_h)
print("flow_g", flow_g)
print("flow_c", flow_c)

flow [[0.63592699]
 [0.06425054]
 [0.23214849]
 [0.93742838]
 [0.89369076]
 [0.37616806]
 [0.35188138]]
flow_h [[ 0.54178346]
 [-1.23320119]
 [ 0.09715402]
 [ 0.71367032]
 [-0.07323464]
 [ 0.20959977]
 [-0.54626241]]
flow_g [[ 0.54178346]
 [-1.23320119]
 [ 0.09715402]
 [ 0.71367032]
 [ 0.07323464]
 [-0.03325359]
 [ 0.25623104]]
flow_c [[ 0.56937773]
 [-0.03325359]
 [-0.25623104]
 [-1.19553723]
 [-0.05868932]
 [ 0.63592699]
 [ 0.06425054]]
